In [1]:
###### Maintenance Script for Updating and Maintaining Chemistry Structure Annotations ######

### Step 1: Initiation ###

from rdkit import Chem, rdBase
from rdkit.Chem import Draw,rdDepictor
from rdkit.Chem import AllChem, Descriptors, DataStructs
from rdkit.Chem.Draw import IPythonConsole, rdMolDraw2D
from rdkit.Chem.Scaffolds import rdScaffoldNetwork
from datetime import datetime
# import math
# import matplotlib as mpl
# from matplotlib import pyplot as plt
# import os
# import pyvis
# from pyvis.network import Network
# import inspect
import psycopg2
from collections import defaultdict
import collections
import sys
sys.path.append('/mnt/raid1/Python_projects')
sys.path.append('/mnt/raid1/Python_projects/pkg_mod')

##### Custom Modules #####
import chemistry_psql as cp
import fingerprinter as fpt

##### Parameters ######

db_host = "192.168.86.31"
#host_main = "localhost"
limit = 2500 # Row limit for SQL 
limit = str(limit)

# DATABASE Connections

# Chemistry Database
#db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
db_chem = psycopg2.connect(host = db_host, dbname="Chemistry", user="postgres", password="postgres")
# create a cursor
cur = db_chem.cursor()
# execute a statement
cur.execute('SELECT version()')
db_ver = cur.fetchone()
print('PostgreSQL database version:', db_ver) 
#display the PostgreSQL database server version


print('RDKit version: ',rdBase.rdkitVersion)
# print('MatplotLib version:', mpl.__version__)
# print('Pyvis version:', pyvis.__version__)
print('Start Time: ',datetime.now())

###################################################################################################################

### Step 2: Generate Fingerprints for New Structures ("no fingerprints") ###


# SELECT Structures without fingerprints

# db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_fingerprint fp WHERE f.structure_id = fp.structure_id)) \
LIMIT '+ limit 
cur.execute(sql) 

mollist = cur.fetchall()
print()
print('##### Structures without Fingerprints #####')
print()
print(len(mollist),'new Structures')
print(mollist)


# generate Morgan fingerprints for molecules

fp_key = {}
fp_val = []
fp_dict = collections.defaultdict(list)
fp_allmol = []
fp_smi_list = {}

for mol in mollist:
    print()
    print('Molecule : ',mol)
    # print(mol[0], mol[1])
    m = Chem.MolFromSmiles(mol[1])

    # Generate Fingerprint SMILES and SMILES list
    fp_smi = fpt.FingerprintToSmiles(m, 3)
    print('Fingerprint : ',fp_smi)    
    for f in fp_smi: 
        fp_smi_list[f[0]] = f[1]
             
    # Generate Fingerprint Footprints:
    big ={}
    fp = AllChem.GetMorganFingerprint(m,3, bitInfo=big)
    # print(len(big))
    # print(big)
    # print(fp)
    for fpb,v in big.items():
        ma = []
        fp_dict[fpb].append(len(v))
        # print(mol[0],fpb, len(v),v)
        ma.append(mol[0])
        ma.append(fpb)
        ma.append(len(v))
        # print(ma)
        fp_allmol.append(ma)
        
print()
print("All Fingerprints by molecule")
print(fp_allmol)
        
print()        
print(fp_smi_list)
print(len(fp_smi_list),'SMILES')

for fpb,v in fp_smi_list.items():
    
    print(fpb,' - ',v)
    
        # key = fbp
        #fp_dict[fpb].append(len(v))
                
print()        
# print(fp_dict)
print(len(fp_dict),'fingerprints')

for fpb,v in fp_dict.items():
    print(fpb,' - ',len(v),' - ',v)
        # key = fbp
        #fp_dict[fpb].append(len(v))
        
###################################################################################################################

### Step 3: Insert new fingerprints into fingerprint table ###

print()
print('##### Insert New Fingerprints #####')
print()

# db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor()
sql = 'SELECT * FROM public.fingerprints WHERE fp_type = 1'
cur.execute(sql)
fpdblist = cur.fetchall()
# print(len(fpdblist), "existing fingerprints")
# print(fpdblist)
# print(fp_smi_list)

print()
if len(fp_smi_list) == 0:
    print('No new fingerprint inserts')
else:
    print("New inserted fingerprints")
print()

sql = 'INSERT INTO public.fingerprints (fp_smiles, fp_number, fp_type) VALUES(%s, %s, 1);'

for fpb,v in fp_smi_list.items():
    smi_fl = 0
    for i in range(0,len(fpdblist)):
        if fpb == fpdblist[i][3]:
            smi_fl = 1

    if smi_fl == 0:
        print(fpb,' - ',v, smi_fl, "inserted")
        cur.execute(sql, (v, fpb))
        db_chem.commit()
        #print(fpb,' - ',v, smi_fl, "inserted")

###################################################################################################################

### Step 4: Insert Fingerprints for New Molecules ###

print()
print('##### Insert Structure/Fingerprints Links #####')
print()

# db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor()
sql = 'SELECT * FROM public.fingerprints WHERE fp_type = 1'
cur.execute(sql)
fpdblist = cur.fetchall()
# print(fpdblist)

fpid = {}

for m in fpdblist:
    # print(m)
    i = m[0]
    j = m[3]
    # print(i,j)
    fpid[j]=i
    
# print(fpid)

cur = db_chem.cursor()
sql = 'INSERT INTO public.structure_fingerprint (structure_id,fp_id,fp_count) VALUES (%s,%s,%s);'

# Insert Notification
print()
if len(fp_allmol) == 0:
    print('No new structure/fingerprint inserts')
else:
    print('New structure/fingerprint links')
print()

for m in fp_allmol:
    # j = m[1]
    i = fpid[m[1]]
    cur.execute(sql, (m[0],i,m[2]))
    db_chem.commit()
    print(m,i)
    
db_chem.close()
    
###################################################################################################################

### Step 5: Update Shannon Entropies ###
print()
print('##### Shannon Update #####')
print()

# This is part of netprepper
#    import pandas as pd
#    from scipy import stats as scistat

# import sys
# print(sys.path)
# set PYTHONPATH to proper directory set holds private modules and packages (here pkg_mod)
import netprepper

# SELECT fingerprint length and append to fp_dict

if len(fp_allmol) == 0:
    print('No new structure/fingerprint inserts')
    print()
else:
    #print('New structure/fingerprint links')
    #db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
    db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
    cur = db_chem.cursor() 
    sql = 'SELECT sf.structure_id, sf.fp_id, sf.fp_count, fp.fp_shannon FROM public.structure_fingerprint sf \
    JOIN public.fingerprints fp ON sf.fp_id = fp.fingerprint_id WHERE fp.fp_type = 1'
    cur.execute(sql)
    fp_list = cur.fetchall()
    # print(fp_list)

    fp_dict = collections.defaultdict(list)
    fp_mol = collections.defaultdict(list)

    for m in fp_list:
        # print(m, m[1],m[2])
        fp_dict[m[1]].append(m[2])
        fp_mol[m[0]].append(m[1])

    # n_fp = 0
    print(len(fp_dict),'fingerprints')
    print(len(fp_mol),'structures')
    print()

    n_fp = len(fp_mol)

    #for fpb, i in fp_dict.items():
    #    n_fp = max(n_fp, len(i))
        # print (fpb, i, n)


    # update entropy field in Fingerprints    
    
    sql = 'UPDATE public.fingerprints SET fp_shannon = %s WHERE fingerprint_id = %s'
    cur = db_chem.cursor()
    for fpb, i in fp_dict.items():
        ns = netprepper.shannon(i,n_fp)
        # print(fpb,' - ',netprepper.shannon(i,n))
        # print(fpb,' - ',ns[0],' - ',ns[1])
        print(fpb,' - ',ns[1])
        cur.execute(sql, (ns[1], fpb))

    db_chem.commit()
    db_chem.close()

###################################################################################################################

### Step 6: Update Tanimoto ###
print()
print('##### Tanimoto Update #####')
print()

db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 

#sql = 'select * FROM public.tanimoto'
#cur.execute(sql)
#q = cur.fetchall()
#print(q)
#print()

sql = 'SELECT max(target) FROM public.tanimoto'
cur.execute(sql)
t = cur.fetchone()
#print(tani_max, type(tani_max))
if t is None:
    tani_max = 0
else:
    tani_max = t[0]
#print(tani_max)

cur = db_chem.cursor() 
sql = 'SELECT structure_id, smiles FROM public.structures ORDER BY structure_id' 
cur.execute(sql) 
mollist = cur.fetchall()
#print(mollist)

cur = db_chem.cursor() 
sql = 'SELECT max(structure_id) FROM public.structures' 
cur.execute(sql) 
m = cur.fetchone()
mol_max = m[0]
print('TaniMax : ',tani_max,' - Mol_max : ',mol_max)

if mol_max > tani_max:
    ml = len(mollist)
    tp = mollist[0][0]
    i = 0
    while tp < tani_max:
        tp = mollist[i][0]
        i = i+1
    print(ml,tp, i)
    tp = i
    cur = db_chem.cursor()
    sql = 'INSERT INTO public.tanimoto (source, target, tanimoto) VALUES (%s,%s,%s);'
    for i in range(tp+1, ml):
        m2 = Chem.MolFromSmiles(mollist[i][1])
        fp2 = AllChem.GetMorganFingerprint(m2,2)
        #fp2 = Chem.RDKFingerprint(m2)
        for j in range (0, i):
            m1 = Chem.MolFromSmiles(mollist[j][1])
            fp1 = AllChem.GetMorganFingerprint(m1,2)
            #fp1 = Chem.RDKFingerprint(m1)
            tan = DataStructs.TanimotoSimilarity(fp1,fp2)
            if tan >= .5:
                #print(i, j, tan, mollist[i][0], mollist[i][1], mollist[j][0], mollist[j][1])
                print(mollist[j][0],mollist[i][0], tan)
                cur.execute(sql, (mollist[j][0],mollist[i][0], tan))

db_chem.commit()
db_chem.close()

##### Tanimoto Backfill #####

db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres") 

cur = db_chem.cursor() 
sql = 'SELECT structure_id, smiles FROM public.structures ORDER BY structure_id' 
cur.execute(sql) 
mollist = cur.fetchall()
#print(mollist)

cur = db_chem.cursor() 
sql = 'select s.structure_id \
from public.structures s \
where s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.tanimoto fp WHERE f.structure_id = fp.target)) \
AND s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.tanimoto fp WHERE f.structure_id = fp.source)) \
ORDER BY s.structure_id' 
cur.execute(sql) 
m = cur.fetchall()
#print(m)

for n in m[1:]:    
    tm = n[0]
    #print(n[0])
    cur = db_chem.cursor() 
    sql = 'SELECT structure_id, smiles FROM public.structures WHERE structure_id <= '+ str(tm) +' ORDER BY structure_id' 
    cur.execute(sql) 
    mollist = cur.fetchall()
    #print(mollist)
    maxtan = 0
    ml = len(mollist)
    #print(tm, ml)
    #tp = mollist[0][0]
    cur = db_chem.cursor()
    sql = 'INSERT INTO public.tanimoto (source, target, tanimoto) VALUES (%s,%s,%s);'
    m2 = Chem.MolFromSmiles(mollist[ml-1][1])
    fp2 = AllChem.GetMorganFingerprint(m2,2)
        #fp2 = Chem.RDKFingerprint(m2)
    for j in range (0, ml-1):
        m1 = Chem.MolFromSmiles(mollist[j][1])
        fp1 = AllChem.GetMorganFingerprint(m1,2)
            #fp1 = Chem.RDKFingerprint(m1)
        tan = DataStructs.TanimotoSimilarity(fp1,fp2)
        if tan > maxtan:
            #print(mollist[j][0],mollist[ml-1][0], tan, maxtan)
            maxtan = tan
            ms = mollist[j][0]
            mt = mollist[ml-1][0]
    if maxtan>0:
        print('Max Tani : ',ms, mt, maxtan)
        cur.execute(sql, (ms,mt, maxtan))
    else:
        print('Tani failed : ', tm)

db_chem.commit()
db_chem.close()

###################################################################################################################

### Step 7: INSERT MISSING CHEMBL ID (if exist) ###

db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
db_chembl = psycopg2.connect(host = "192.168.86.31", dbname="chembl_28", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_properties fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 4))' 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing ChemBL_ID #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    m = Chem.MolFromSmiles(j)
    inchi = Chem.inchi.MolToInchiKey(m, options='')
    #print(i, inchi)
    cur = db_chembl.cursor()
    sql = 'SELECT cs.molregno, md.chembl_id FROM public.compound_structures cs \
    JOIN public.molecule_dictionary md ON cs.molregno = md.molregno \
    WHERE cs.standard_inchi_key = \''+inchi+'\''
    #print(sql)
    cur.execute(sql)
    chmol = cur.fetchall()
    if len(chmol) == 0:
        #print()
        print('No CHEMBL: ', i, inchi)
        #print()
    else:
        print(i, chmol)
        a = (4, i[0], chmol[0][1])
        #print(a)
        cp.StructPropInsert(a)
        
db_chembl.commit()

###################################################################################################################

### Step 8: INSERT MISSING INCHI KEYS ###

db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_properties fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 5)) LIMIT '+ limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing InChi Keys #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    m = Chem.MolFromSmiles(j)
    inchi = Chem.inchi.MolToInchiKey(m, options='')
    a = (5, i[0], inchi)
    #print(a)
    cp.StructPropInsert(a)
        
db_chem.commit()

###################################################################################################################

### Step 9: INSERT MISSING QED ###

db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_prop_calc fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 1)) LIMIT ' + limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing QED #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    m = Chem.MolFromSmiles(j)
    qed = Chem.QED.default(m)
    a = (1, i[0], qed)
    #print(a)
    cp.StructCalcPropInsert(a)
        
db_chem.commit()

###################################################################################################################

### Step 9: INSERT MISSING LOGP ###

db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_prop_calc fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 2)) LIMIT ' + limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing LogP #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    m = Chem.MolFromSmiles(j)
    #qed = Chem.QED.default(m)
    log = Chem.Crippen.MolLogP(m, True)
    a = (2, i[0], log)
    #print(a)
    cp.StructCalcPropInsert(a)
        
db_chem.commit()

###################################################################################################################

### Step 10: INSERT MISSING FSP3 ###

db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_prop_calc fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 6)) LIMIT ' + limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing Fsp3 #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    m = Chem.MolFromSmiles(j)
    fsp = Chem.Lipinski.FractionCSP3(m)
    a = (6, i[0], fsp)
    #print(a, j)
    cp.StructCalcPropInsert(a)
        
db_chem.commit()

###################################################################################################################

### Step 11: INSERT MISSING H-DONOR ###

db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_prop_calc fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 7)) LIMIT ' + limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing H-Donor #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    m = Chem.MolFromSmiles(j)
    hdon = Chem.Lipinski.NumHDonors(m)
    a = (7, i[0], hdon)
    #print(a, j)
    cp.StructCalcPropInsert(a)
        
db_chem.commit()

###################################################################################################################

### Step 12: INSERT MISSING H-ACCEPTOR ###

db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_prop_calc fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 8)) LIMIT ' + limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing H-ACCEPTOR #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    m = Chem.MolFromSmiles(j)
    hacc = Chem.Lipinski.NumHAcceptors(m)
    a = (8, i[0], hacc)
    print(a, j)
    cp.StructCalcPropInsert(a)
        
db_chem.commit()

###################################################################################################################

### Step 13: INSERT MISSING MOLECULAR WEIGHT ###

db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_prop_calc fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 9)) LIMIT ' + limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing Mol Weights #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    m = Chem.MolFromSmiles(j)
    mw = round(Chem.Descriptors.MolWt(m), 3)    
    a = (9, i[0], mw)
    #print(a, j)
    cp.StructCalcPropInsert(a)
        
db_chem.commit()

###################################################################################################################

### Step 14: INSERT MISSING LIPINSKI ###

db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s.structure_id, s.smiles FROM public.structures s \
WHERE s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM public.structure_prop_calc fp WHERE f.structure_id = fp.structure_id \
AND fp.property_type_id = 10)) LIMIT ' + limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing Rule of Five Counts #####')
#print(mollist)
print()

for i in mollist:
    j= i[1]
    lip =0
    m = Chem.MolFromSmiles(j)
    mw = round(Chem.Descriptors.MolWt(m), 3)
    log = Chem.Crippen.MolLogP(m, True)
    hacc = Chem.Lipinski.NumHDonors(m)
    hdon = Chem.Lipinski.NumHDonors(m)
    if mw > 500:
        lip = lip + 1
    if log > 5:
        lip = lip + 1
    if hacc > 10:
        lip = lip + 1
    if hdon > 5:
        lip = lip + 1
    a = (10, i[0], lip)
    #print(mw, log, hacc, hdon)
    #print(a, j)
    cp.StructCalcPropInsert(a)
        
db_chem.commit()

###################################################################################################################

### Step 15: INSERT MISSING OBJECTS FOR STRUCTURES AND UPDATE STRUCTURE TABLE ###

db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'select s.structure_id \
from public.structures s \
where s.structure_id IN (SELECT f.structure_id FROM public.structures f \
WHERE NOT EXISTS (SELECT 1 FROM zafrens.objects fp WHERE f.structure_id = fp.obj_ref \
AND fp.obj_class = \'s\')) LIMIT ' + limit 
cur.execute(sql) 

mollist = cur.fetchall()

print()
print('##### Missing Structure Objects #####')
#print(mollist)
#print()

sql = "INSERT INTO zafrens.objects(obj_ref, obj_class) VALUES("+j+", 's');"
for i in mollist:
    j= str(i[0])
    #print(j)
    sql = "INSERT INTO zafrens.objects(obj_ref, obj_class) VALUES("+j+", 's');"
    print(sql)
    cur.execute(sql)
        
db_chem.commit()

sql = "select z.obj_id, s.structure_id from public.structures s  join zafrens.objects z \
on s.structure_id = z.obj_ref and z.obj_class = 's' and s.obj_id is NULL"
cur.execute(sql)
objlist = cur.fetchall()
print(objlist)

sql = "update public.structures s set obj_id = z.obj_id from zafrens.objects z \
where structure_id = z.obj_ref and z.obj_class = 's' and s.obj_id is NULL"
cur.execute(sql) 
    
db_chem.commit()


###############################################################################################################
print()
print('#####     #####     #####')
print()
print('Finish Time: ',datetime.now())

PostgreSQL database version: ('PostgreSQL 12.8 (Ubuntu 12.8-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit',)
RDKit version:  2020.03.3
Start Time:  2021-10-11 23:28:13.374711

##### Structures without Fingerprints #####

0 new Structures
[]

All Fingerprints by molecule
[]

{}
0 SMILES

0 fingerprints

##### Insert New Fingerprints #####


No new fingerprint inserts


##### Insert Structure/Fingerprints Links #####


No new structure/fingerprint inserts


##### Shannon Update #####

Net_Prepper_Imported
No new structure/fingerprint inserts


##### Tanimoto Update #####

TaniMax :  16281  - Mol_max :  16281

##### Missing ChemBL_ID #####

No CHEMBL:  (38, 'CCCNCC') XCVNDBIXFPGMIW-UHFFFAOYSA-N
No CHEMBL:  (62, 'C[C@H](C(C1=NC=C(C=C1)O)O)C(C(=O)NC(C2C(C(C(O2)N3C=CC(=O)NC3=O)O)O)C(=O)O)N') WWJFFVUVFNBJTN-PQHFPETQSA-N
No CHEMBL:  (98, 'O=C1C2NC(C)NC2NC(N)N1') WXTLCUCBYQGXCL-UHFFFAOYSA-N
No CHEMBL:  (72, 'C1C=CSC=C1') STBGWNRZMPCVTG-UHFFF

(8, 307, 8) Cc1cn2c(-c3cn[nH]c3)cnc2c(Nc2cc(CN3CC[C@@H](F)C3)ns2)n1.Cl
(8, 308, 8) Cc1cn2c(-c3cn[nH]c3)cnc2c(Nc2cc(CN3CCC(F)(F)C3)ns2)n1.Cl
(8, 309, 8) Cc1cn2c(-c3cn[nH]c3)cnc2c(Nc2cc(CN3CC(F)(F)C(F)(F)C3)ns2)n1.Cl
(8, 310, 9) Cc1cn2c(-c3cn[nH]c3)cnc2c(Nc2cc(CN3CC(F)C3)ns2)n1.O=C(O)C(F)(F)F
(8, 311, 9) Cc1cn2c(-c3cn[nH]c3)cnc2c(Nc2cc(CN3CC(F)(F)C3)ns2)n1.O=C(O)C(F)(F)F
(8, 312, 8) Cc1cn2c(-c3cn[nH]c3)cnc2c(Nc2cc(CNCCC(F)(F)F)ns2)n1.Cl
(8, 313, 8) Cc1cn2c(-c3cn[nH]c3)cnc2c(Nc2cc(CNCC(F)(F)C(F)(F)F)ns2)n1.Cl
(8, 314, 8) Cl.[2H]C([2H])(c1cc(Nc2nc(C)cn3c(-c4cn[nH]c4)cnc23)sn1)N1CCC(F)(F)C1
(8, 315, 4) Cc1cc(Nc2n[nH]c(=O)c3ccccc23)n[nH]1
(8, 316, 5) Cc1cc(Nc2nn(Cc3ccc(F)cc3)c(=O)c3ccccc23)n[nH]1
(8, 317, 7) Cc1cc(Nc2nn(Cc3ccc(S(C)(=O)=O)cc3)c(=O)c3ccccc23)n[nH]1
(8, 318, 6) COc1ccc(Cn2nc(Nc3cc(C)[nH]n3)c3ccccc3c2=O)cc1
(8, 319, 6) COc1cccc(Cn2nc(Nc3cc(C)[nH]n3)c3ccccc3c2=O)c1
(8, 320, 6) CC(=O)Nc1ccc(Cn2nc(Nc3cc(C)[nH]n3)c3ccccc3c2=O)cc1
(8, 321, 6) Cc1cc(Nc2nn(Cc3ccncc3)c(=O)c3ccccc23)n[nH

(8, 587, 5) CNC(=O)c1cnc(N)c2c(-c3ccc(NC(=O)Nc4cccc(C)c4)cc3)csc12
(8, 588, 5) Cc1cccc(NC(=O)Nc2ccc(-c3csc4c(-c5ccoc5)cnc(N)c34)cc2)c1
(8, 589, 5) Cc1cccc(NC(=O)Nc2ccc(-c3csc4c(-c5ccsc5)cnc(N)c34)cc2)c1
(8, 590, 5) Cc1cccc(NC(=O)Nc2ccc(-c3csc4c(-c5cc[nH]n5)cnc(N)c34)cc2)c1
(8, 591, 6) Cc1ccccc1NC(=O)Nc1ccc(-c2csc3c(-c4cnn(C)c4)cnc(N)c23)cc1
(8, 592, 6) Cn1cc(-c2cnc(N)c3c(-c4ccc(NC(=O)Nc5ccccc5)cc4)csc23)cn1
(8, 593, 6) Cn1cc(-c2cnc(N)c3c(-c4ccc(NC(=O)Nc5ccccc5F)cc4)csc23)cn1
(8, 594, 6) Cn1cc(-c2cnc(N)c3c(-c4ccc(NC(=O)Nc5cccc(Cl)c5)cc4)csc23)cn1
(8, 595, 6) Cn1cc(-c2cnc(N)c3c(-c4ccc(NC(=O)Nc5ccc(Cl)cc5)cc4)csc23)cn1
(8, 596, 7) COc1ccccc1NC(=O)Nc1ccc(-c2csc3c(-c4cnn(C)c4)cnc(N)c23)cc1
(8, 597, 7) COc1ccc(NC(=O)Nc2ccc(-c3csc4c(-c5cnn(C)c5)cnc(N)c34)cc2)cc1
(8, 598, 7) Cn1cc(-c2cnc(N)c3c(-c4ccc(NC(=O)Nc5ccc(OC(F)(F)F)cc5)cc4)csc23)cn1
(8, 599, 10) Cc1cn2c(-c3cnn(Cc4nc(-c5ccccc5)c[nH]4)c3)cnc2c(Nc2cc(CN3CCCCC3)ns2)n1
(8, 600, 10) Cc1cn2c(-c3cnn(CC(=O)NCc4ccccc4F)c3)cnc2c(Nc2cc(CN3CCCCC3)n

(8, 846, 5) CNC(=O)c1cccc2cc(Oc3ccnc4cc(OC)c(OC)cc34)ccc12
(8, 847, 5) COc1cc2nccc(Oc3ccc4c(C(=O)NCC5CC5)cccc4c3)c2cc1OC
(8, 848, 6) COCCNC(=O)c1cccc2cc(Oc3ccnc4cc(OC)c(OC)cc34)ccc12
(8, 849, 5) COc1cc2nccc(Oc3ccc4c(C(N)=O)cccc4c3)c2cc1OC
(8, 850, 5) COc1cc2nccc(Oc3ccc4c(C(=O)NC5CC5)cccc4c3)c2cc1OC
(8, 851, 5) COc1cc2nccc(Oc3ccc4c(C(=O)NC5CC5)cccc4c3)c2cc1C(N)=O
(8, 852, 4) COc1ccc2c(Oc3ccc4c(C(N)=O)cccc4c3)ccnc2c1
(8, 853, 2) Cc1[nH]c(/C=C2\C(=O)Nc3ccccc32)c(C)c1CCC(=O)O
(8, 854, 8) Cc1nc(N)sc1-c1ccnc(Nc2cccc([N+](=O)[O-])c2)n1
(8, 855, 5) COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OC
(8, 856, 8) Cc1cc(NC(=O)c2ccc(Oc3c(Cl)cnc4[nH]c(-c5cn(C)nc5C)nc34)cc2)on1
(8, 857, 5) Cn1cc(-c2[nH]c3ncc(Cl)cc3c2-c2cccc(CNS(C)(=O)=O)c2)cn1
(8, 858, 4) COc1ccc(-c2[nH]c3ncc(Cl)cc3c2-c2cccc(CNS(C)(=O)=O)c2)cc1
(8, 859, 4) CC(=O)NCc1cccc(-c2c(-c3cnn(C)c3)[nH]c3ncc(Cl)cc23)c1
(8, 860, 3) COc1ccc(-c2[nH]c3ncc(Cl)cc3c2-c2cccc(CNC(C)=O)c2)cc1
(8, 861, 6) CC(=O)N[C@H]1CCN(c2c(Cl)cnc3[nH]c(-c4cn(C)nc4C)nc23)C1
(8, 862, 

(8, 1114, 4) CN1CCN(c2ccc(C(=O)Nc3n[nH]c4ccc(Cc5cc(F)cc(F)c5)cc34)cc2)CC1
(8, 1115, 6) COCCNc1cc(N2CCN(C)CC2)ccc1C(=O)Nc1n[nH]c2ccc(Cc3cc(F)cc(F)c3)cc12
(8, 1116, 5) CN1CCN(c2ccc(C(=O)Nc3n[nH]c4ccc(Cc5cc(F)cc(F)c5)cc34)c(NC3CCCCC3)c2)CC1
(8, 1117, 7) NC(=O)c1cc(OCCCN2CCC(O)C2)cc2c(NCc3ccc(C(F)(F)F)cc3)ncnc12
(8, 1118, 7) NC(=O)c1cc(OCCCN2CCC[C@H]2CO)cc2c(NCc3ccc(C(F)(F)F)cc3)ncnc12
(8, 1119, 7) CN1CCN(CCCOc2cc(C(N)=O)c3ncnc(NCc4ccc(C(F)(F)F)cc4)c3c2)CC1
(8, 1120, 7) NC(=O)c1cc(OCCCNCCO)cc2c(NCc3ccc(C(F)(F)F)cc3)ncnc12
(8, 1121, 6) COCCCOc1cc(C(N)=O)c2ncnc(NCc3ccc(C(F)(F)F)cc3)c2c1
(8, 1122, 6) COc1ccccc1C(CN)Nc1ncnc2c(C(N)=O)cccc12
(8, 1123, 6) COc1ccccc1C(CNC(C)=O)Nc1ncnc2c(C(N)=O)cccc12
(8, 1124, 4) NC(=O)c1cccc2c(NCc3cccc(F)c3)ncnc12
(8, 1125, 4) NC(=O)c1cccc2c(NCc3cccc4ccccc34)ncnc12
(8, 1126, 4) NC(=O)c1cccc2c(NCc3ccccc3C(F)(F)F)ncnc12
(8, 1127, 6) NC(=O)c1cccc2c(NCc3ccc4c(c3)OCO4)ncnc12
(8, 1128, 4) NC(=O)c1cccc2c(NCc3ccc(C(F)(F)F)cc3)ncnc12
(8, 1129, 5) NC(=O)c1cccc2c(NCc3cccc(O

...

(9, 2009, 0) O=c1[nH]c(-c2ccc(O)cc2)nc2sc3c(c12)CCCCC3
441.466 4.490820000000004 1 1
(9, 2010, 0) Cc1nnc2n1-c1ccccc1C(c1ccccc1)=NC2NC(=O)OCc1ccc(F)cc1
427.439 4.353020000000003 1 1
(9, 2011, 0) Cc1nnc2n1-c1ccccc1C(c1ccccc1)=NC2NC(=O)Oc1cccc(F)c1
361.405 3.1714200000000012 1 1
(9, 2012, 0) CCOC(=O)NC1N=C(c2ccccc2)c2ccccc2-n2c(C)nnc21
361.405 3.1714200000000012 1 1
(9, 2013, 0) CCOC(=O)N[C@H]1N=C(c2ccccc2)c2ccccc2-n2c(C)nnc21
361.405 3.1714200000000012 1 1
(9, 2014, 0) CCOC(=O)N[C@@H]1N=C(c2ccccc2)c2ccccc2-n2c(C)nnc21
395.85 3.8248200000000026 1 1
(9, 2015, 0) CCOC(=O)NC1N=C(c2ccc(Cl)cc2)c2ccccc2-n2c(C)nnc21
489.938 5.297720000000004 1 1
(9, 2016, 1) COc1ccc2c(c1)C(c1ccc(Cl)cc1)=N[C@@H](CC(=O)Nc1ccc(F)cc1)c1nnc(C)n1-2
477.977 5.220120000000004 1 1
(9, 2017, 1) COc1ccc2c(c1)C(c1ccc(Cl)cc1)=N[C@@H](CC(=O)Nc1cccs1)c1nnc(C)n1-2
472.936 4.553620000000004 1 1
(9, 2018, 0) COc1ccc2c(c1)C(c1ccc(Cl)cc1)=N[C@@H](CC(=O)Nc1ccccn1)c1nnc(C)n1-2
472.936 4.553620000000004 1 1
(9, 2019, 0) COc1ccc2c(c1)C

305.377 4.140540000000003 1 1
(9, 2121, 0) Cc1noc(C)c1-c1ccc2c(c1)C(O)(c1ccccc1)CC2
321.355 4.495540000000003 0 0
(9, 2122, 0) Cc1noc(C)c1-c1ccc2ncn(Cc3cccc(F)c3)c2c1
337.81 5.009840000000003 0 0
(9, 2123, 1) Cc1noc(C)c1-c1ccc2ncn(Cc3cccc(Cl)c3)c2c1
337.81 5.009840000000003 0 0
(9, 2124, 1) Cc1noc(C)c1-c1ccc2ncn(Cc3ccc(Cl)cc3)c2c1
337.81 5.009840000000003 0 0
(9, 2125, 1) Cc1noc(C)c1-c1ccc2c(c1)ncn2Cc1ccc(Cl)cc1
382.261 5.118940000000003 0 0
(9, 2126, 1) Cc1noc(C)c1-c1ccc2ncn(Cc3cccc(Br)c3)c2c1
382.261 5.118940000000003 0 0
(9, 2127, 1) Cc1noc(C)c1-c1ccc2c(c1)ncn2Cc1cccc(Br)c1
333.391 4.365040000000003 0 0
(9, 2128, 0) COc1ccc(Cn2cnc3ccc(-c4c(C)noc4C)cc32)cc1
337.81 5.009840000000002 0 0
(9, 2129, 1) Cc1noc(C)c1-c1ccc2ncn(Cc3ccccc3Cl)c2c1
348.362 4.264640000000003 0 0
(9, 2130, 0) Cc1noc(C)c1-c1ccc2ncn(Cc3ccccc3[N+](=O)[O-])c2c1
348.362 4.264640000000003 0 0
(9, 2131, 0) Cc1noc(C)c1-c1ccc2c(c1)ncn2Cc1ccccc1[N+](=O)[O-]
328.375 4.228120000000002 0 0
(9, 2132, 0) Cc1noc(C)c1-c1ccc2ncn(Cc

323.355 1.8921999999999999 2 2
(9, 2232, 0) O=c1[nH]c(-c2cccc(S(=O)(=O)Nc3ccon3)c2)cs1
340.426 1.5606999999999995 2 2
(9, 2233, 0) O=c1[nH]c(-c2cccc(S(=O)(=O)NCC3CCCO3)c2)cs1
326.399 1.1705999999999996 2 2
(9, 2234, 0) O=c1[nH]c(-c2cccc(S(=O)(=O)NC3CCOC3)c2)cs1
338.454 2.632720000000001 2 2
(9, 2235, 0) Cc1ccc(-c2csc(=O)[nH]2)cc1S(=O)(=O)NC1CCCC1
354.453 2.3329000000000004 2 2
(9, 2236, 0) COc1ccc(-c2csc(=O)[nH]2)cc1S(=O)(=O)NC1CCCC1
358.872 2.9777000000000013 2 2
(9, 2237, 0) O=c1[nH]c(-c2ccc(Cl)c(S(=O)(=O)NC3CCCC3)c2)cs1
342.417 2.463400000000001 2 2
(9, 2238, 0) O=c1[nH]c(-c2ccc(F)c(S(=O)(=O)NC3CCCC3)c2)cs1
339.442 1.9065 3 3
(9, 2239, 0) Nc1ccc(-c2csc(=O)[nH]2)cc1S(=O)(=O)NC1CCCC1
411.505 1.9091999999999996 3 3
(9, 2240, 0) COCC(=O)Nc1ccc(-c2csc(=O)[nH]2)cc1S(=O)(=O)NC1CCCC1
407.517 2.6728000000000014 3 3
(9, 2241, 0) O=C(Nc1ccc(-c2csc(=O)[nH]2)cc1S(=O)(=O)NC1CCCC1)C1CC1
403.323 3.086800000000002 2 2
(9, 2242, 0) O=c1[nH]c(-c2cc(Br)cc(S(=O)(=O)NC3CCCC3)c2)cs1
417.35 3.3952200000000

386.496 1.9416199999999992 4 4
(9, 2355, 0) Cc1cc2ccnc(N[C@@H]3CCNC[C@H]3OC[C@H]3CC[C@H](O)CC3)c2[nH]c1=O
372.469 1.8171199999999996 3 3
(9, 2356, 0) Cc1cc2ccnc(N[C@@H]3CCNC[C@H]3OCC3CCOCC3)c2[nH]c1=O
372.469 1.8171199999999996 3 3
(9, 2357, 0) Cc1cc2ccnc(N[C@H]3CCNC[C@@H]3OCC3CCOCC3)c2[nH]c1=O
420.535 1.21532 3 3
(9, 2358, 0) Cc1cc2ccnc(N[C@@H]3CCNC[C@H]3OCC3CCS(=O)(=O)CC3)c2[nH]c1=O
420.535 1.21532 3 3
(9, 2359, 0) Cc1cc2ccnc(N[C@H]3CCNC[C@@H]3OCC3CCS(=O)(=O)CC3)c2[nH]c1=O
406.508 0.8252200000000003 3 3
(9, 2360, 0) Cc1cc2ccnc(N[C@@H]3CCNC[C@H]3OC[C@@H]3CCS(=O)(=O)C3)c2[nH]c1=O
288.351 1.0203199999999997 3 3
(9, 2361, 0) CO[C@@H]1CNCC[C@H]1Nc1nccc2cc(C)c(=O)[nH]c12
316.405 1.7989199999999999 3 3
(9, 2362, 0) Cc1cc2ccnc(N[C@@H]3CCNC[C@H]3OC(C)C)c2[nH]c1=O
316.405 1.8005199999999995 3 3
(9, 2363, 0) CCCO[C@@H]1CNCC[C@H]1Nc1nccc2cc(C)c(=O)[nH]c12
330.432 2.0465199999999997 3 3
(9, 2364, 0) Cc1cc2ccnc(N[C@@H]3CCNC[C@H]3OCC(C)C)c2[nH]c1=O
356.47 2.5807200000000003 3 3
(9, 2365, 0) Cc1cc2c

472.94 3.9769600000000027 0 0
(9, 2466, 0) Cc1nn(-c2ccnn2C)c2c1C(c1ccc(Cl)cc1)N(c1cc(C)c3nnc(C)n3c1)C2=O
432.915 4.589160000000004 0 0
(9, 2467, 0) Cc1nn(C2CC2)c2c1C(c1ccc(Cl)cc1)N(c1cc(C)c3nnc(C)n3c1)C2=O
432.915 4.589160000000004 0 0
(9, 2468, 0) Cc1cc(N2C(=O)c3nn(C4CC4)c(C)c3C2c2ccc(Cl)cc2)cn2c(C)nnc12
422.916 4.326840000000004 0 0
(9, 2469, 0) Cc1nn(C2CCC2)c2c1C(c1ccc(Cl)cc1)N(c1cc(C)c(=O)n(C)c1)C2=O
458.44 4.1819400000000035 0 0
(9, 2470, 0) Cc1nn(C2CC2)c2c1C(c1ccc(OC(F)(F)F)cc1)N(c1cc(C)c(=O)n(C)c1)C2=O
442.441 4.302140000000003 0 0
(9, 2471, 0) Cc1nn(C2CC2)c2c1C(c1cccc(C(F)(F)F)c1)N(c1cc(C)c(=O)n(C)c1)C2=O
442.441 4.302140000000003 0 0
(9, 2472, 0) Cc1nn(C2CC2)c2c1C(c1ccc(C(F)(F)F)cc1)N(c1cc(C)c(=O)n(C)c1)C2=O
506.95 3.398240000000002 0 0
(9, 2473, 1) COc1ncc(-n2nc(C)c3c2C(=O)N(c2cc(C)c(=O)n(C)c2)[C@H]3c2ccc(Cl)cc2)c(OC)n1
506.95 3.398240000000002 0 0
(9, 2474, 1) COc1ncc(-n2nc(C)c3c2C(=O)N(c2cc(C)c(=O)n(C)c2)[C@@H]3c2ccc(Cl)cc2)c(OC)n1
499.962 4.647060000000004 0 0
(9, 2475, 0)